# Modelagem de dispersão

Nesta atividade vocês precisarão implementar o modelo gaussiano no Python. Vocês devem realizar os seguintes tópicos:

- Implementar uma função para determinar a classe de estabilidade de Pasquil para diferentes condições atmosféricas.

- Implementar a função de estimativa de coefiente de dispersão (sigmaYZ) para todas as classes de estabilidade.

- Implementar a função de estimativa de sobrelevação da pluma utilizando os métodos de Davidson-Bryant, Holland e Briggs (quem fizer o método de Briggs ganha um ponto a mais). Deve ser considerado o efeito Tip-Downwash

- Implementar a função do modelo gaussiano

- Realizar simulações com o script criado, utilizando diferentes classes de estabilidade, velocidades do vento, alturas de chaminé. Considere a taxa de emissão que você estimou na primeira atividade. Encontre a altura de chaminé necessária para que as concentrações não violem os padrões da Resolução CONAMA 491.

- Faça figuras e discuta os resultados.

In [9]:
# Criando função de classe de estabilidade de Pasquil

def classePasquill(v,I,cco,cob):
    if cob == 1: # Valor 1 para quando tem cobertura
        classe = 'D'
    elif v < 2:
        if I <= 0:
            classe = 'F' # Não sei qual colocar
        elif I > 0 and I < 350:
            classe = 'B'
        elif I >= 350 and I < 700:
            classe = 'B' # Ou A
        else:
            classe = 'A'
    elif v >= 2 and v < 3:
        if I <= 0:
            if cco >=4:
                classe = 'E'
            else:
                classe = 'F'
        elif I > 0 and I < 350:
            classe = 'C'
        elif I >= 350 and I < 700:
            classe = 'B'
        else:
            classe = 'B' # Ou A
    elif v >= 3 and v < 5:
        if I <= 0:
            if cco >=4:
                classe = 'D'
            else:
                classe = 'E'
        elif I > 0 and I < 350:
            classe = 'C'
        elif I >= 350 and I < 700:
            classe = 'C' # Ou B
        else:
            classe = 'B'
    elif v >= 5 and v < 6:
        if I <= 0:
            if cco >=4:
                classe = 'D'
            else:
                classe = 'D'
        elif I > 0 and I < 350:
            classe = 'D'
        elif I >= 350 and I < 700:
            classe = 'C' # Ou D
        else:
            classe = 'C'
    else:
        if I <= 0:
            if cco >=4:
                classe = 'D'
            else:
                classe = 'D'
        elif I > 0 and I < 350:
            classe = 'D'
        elif I >= 350 and I < 700:
            classe = 'D'
        else:
            classe = 'C'
    return classe

In [6]:
# Criando função da estimativa da dispersão lateral e vertical da pluma
def sigmaXY(x,classe,urbOrRural):
    if urbOrRural =='urbano':
        if classe == 'A' or classe == 'B':
            sigmaY = 0.32*x*(1+0.0004*x)**(-0.5) 
            sigmaZ = 0.24*x*(1+0.001*x)**(0.5)
        elif classe == 'C':
            sigmaY = 0.22*x*(1+0.0004*x)**(-0.5) 
            sigmaZ = 0.20*x
        elif classe == 'D':
            sigmaY = 0.16*x*(1+0.0004*x)**(-0.5) 
            sigmaZ = 0.14*x*(1+0.0003*x)**(-0.5)
        elif classe == 'E' or classe == 'F':
            sigmaY = 0.11*x*(1+0.0004*x)**(-0.5) 
            sigmaZ = 0.08*x*(1+0.0015*x)**(-0.5)
        else:
            print('Classe de estabilidade errada.')
    else:
        if classe == 'A':
            sigmaY = 0.22*x*(1+0.0001*x)**(-0.5) 
            sigmaZ = 0.20*x
        elif classe == 'B':
            sigmaY = 0.16*x*(1+0.0001*x)**(-0.5) 
            sigmaZ = 0.12*x
        elif classe == 'C':
            sigmaY = 0.11*x*(1+0.0001*x)**(-0.5) 
            sigmaZ = 0.08*x*(1+0.0002*x)**(-0.5)
        elif classe == 'D':
            sigmaY = 0.08*x*(1+0.0001*x)**(-0.5) 
            sigmaZ = 0.06*x*(1+0.0015*x)**(-0.5)
        elif classe == 'E':
            sigmaY = 0.06*x*(1+0.0001*x)**(-0.5) 
            sigmaZ = 0.03*x*(1+0.0003*x)**(-1)
        elif classe == 'F':
            sigmaY = 0.04*x*(1+0.0001*x)**(-0.5) 
            sigmaZ = 0.016*x*(1+0.0003*x)**(-1)
        else:
            print('Classe de estabilidade errada.')
    return sigmaY,sigmaZ

In [22]:
# Função para estimar o deltaH com base na equação de Davidson-Bryant
def deltaHdavidsonbryant(d,vs,u,Ts,Tamb):
    deltaH = (d*(vs/u)**(1.4))*(1+(Ts-Tamb)/Tamb)
    return deltaH

def deltaHholland(d,vs,u,Ts,Tamb,p,classe):
    if classe == 'A':
        ac = 1.2
    elif classe == 'B':
        ac = 1.1
    elif classe == 'C' or classe == 'D':
        ac = 1
    elif classe == 'E':
        ac = 0.9
    elif classe == 'F':
        ac = 0.8
    else:
        print('Classe de estabilidade errada')
    deltaH = ac*(vs*d/u)*(1.5+(2.68*p*(Ts-Tamb)*d)/(Ts*1000))
    return deltaH

def deltaHbriggs(d,vs,Ts,Tamb,classe,):
    Fb = 9.8*((d/2)**2)*vs*(1-Tamb/Ts)
    dodz = 0.02 ###########################################################CORRRIGGGIRRRR
    s = (9.8/Tamb)*(dodz)
    if classe == 'A' or classe == 'B' or classe == 'C' or classe == 'D':
        if Fb < 55:
            deltaTc = 0.0297*Ts*(vs**(1/3))/(d**(2/3))
            if deltaTc > (Ts - Tamb):
                deltaH = 3*d*vs/u
            else:
                deltaH = 21.425*Fb**(3/4)/u
        else:
            deltaTc = 0.0057*Ts*(vs**(2/3)/d**(1/3))
            if deltaTc > (Ts - Tamb):
                deltaH = 3*d*vs/u
            else:
                deltaH = 2.6*(Fb/(u*s))**(1/3)
    elif classe == 'E' or classe == 'F':
        deltaTc = 0.019582*Ts*vs*s**(1/2)
        if deltaTc > (Ts - Tamb):##################OQUEFAZERRRR###################
            Fm = (vs**2)*(d**2)*Tamb/(4*Ts) 
            deltaH1 = 1.5*(Fm/(u*s**(1/2)))**(1/3)
            deltaH2 = 3*d*vs/u
            if deltaH1 > deltaH2:
                deltaH = deltaH2
            else:
                deltaH = deltaH1
        else:
            deltaH = 2.6*(Fb/(u*s))**(1/3)
    return deltaH

In [8]:
#Criando uma função do modelo gaussiano
import numpy as np
def modeloGaussiano(qs,sigmaY,sigmaZ,u,y,z,H):
    termo1 = qs/(2*np.pi*sigmaY*sigmaZ*u)
    termo2 = np.exp((-y**2)/(2*sigmaY**2))
    termo3 = np.exp((-(z-H)**2)/(2*sigmaZ**2)) + np.exp((-(z+H)**2)/(2*sigmaZ**2))
    conc = termo1*termo2*termo3
    conc = conc*10**6
    return conc